# Programming for Data Analysis - Project


## Abstract


In this project, you will investigate and explain box plots and their uses.

In your notebook, you should:
• Summarise the history of the box plot and situations in which it used.
• Demonstrate the use of the box plot using data of your choosing.
• Explain any relevant terminology such as the terms quartile and percentile.
• Compare the box plot to alternatives.

Box plot is one of the many tools used during exploratory data analysis, as it conveys a multitude of information visually. Its history stems from the work by John W. Tukey in 1969 that was further popularized by his formal publication in 1977. In this Jupyter Python notebook, the Road Accident Deaths in US States dataset is examined by box plots. Finally, a comparison of box plot to several other types of plots is also made to illustrate the advantages and disadvantages of box plot. All the plots herein are fully interactive, allowing the user to zoom in/out, rescale the axis and edit them with the Plotly Chart Studio. 


http://vita.had.co.nz/papers/boxplots.pdf




## Table of Contents

1. [Introduction](#introduction)
1. [Exploratory Data Analysis (EDA)](#eda)
1. [Methods for Data Synthesis / Simulation](#methodsForSimulation)
1. [Generation of a synthetic dataset](#generationOfSyntheticDataset)
1. [RMSD](#RMSD)
1. [Total Surface Area](#TotalArea)
1. [Discussion](#discussion)
1. [Conclusion](#conclusion)


## <a id='introduction'></a>Introduction - History of Box Plot and Its Uses

Prior to performing more computationally-expensive analysis such as machine learning, it is important to visually and numerically explore the dataset in question. There are several approaches that can be taken, including [boxplots](https://en.wikipedia.org/wiki/Box_plot), violin plots, correlation matrices, histograms and kernel density estimates - these are not mutually exclusive and can be achieved by simple commands. These approaches can be further augmented by interactive plots, enabled by Plotly and Bokeh libraries. The figure below summarizes the different Python plotting libraries.

<img src='Python-plot-tools.png' width='500'>
<div align='right'><i>Taken from <a href='http://pbpython.com/python-vis-flowchart.html'>PbPython</i></a></div>

As the name suggests, box plots consist of a rectangle with a band within, as shown by the figure below.

<img src='boxplot_without_whiskers.png' width='200'>
<div align='right'><i>Taken from a <a href='https://stackoverflow.com/questions/18459287/remove-whiskers-in-box-whisker-plot'>StackOverflow post</i></a></div>

More often that not, this rectangle is also extended by lines ('whiskers'), leading these plots to be termed box and whisker plots. The anatomy of a box and whisker plot (hereafter, the term 'box plot' will be used) is described by the following figure:

<img src='simple_box_defs.gif' width='300'>
<div align='right'><i>Taken from <a href='http://www.physics.csbsju.edu/stats/box2.html'>Statistics to Use</i></a></div>

1. Minimum and maximum - these are the minimum and maximum values for a given dataset and are represented by the whiskers.
1. A dataset can be divided into quarters (4 parts) or percentiles (100 parts). These are collectively known as [quantiles](https://en.wikipedia.org/wiki/Quantile), and a dataset can be split in more ways than these two.
    1. When split into two equal portions, **median is the value in the middle** (for odd number of data points) or the average of the two middle data points (for even number of data points). The data has to be sorted in increasing order first.
<img src='Finding_the_median.png' width='200'>
<div align='right'><i>Taken from <a href='https://en.wikipedia.org/wiki/Median'>Wikipedia: Median</i></a></div><br>
    1. The first half can then be further split into two, with the 'median' of this subset being the **first quartile**, representing the first quarter or 25 % of the dataset. In other words, 25 % of the values lie below the first quartile, and the rest are above it. Similarly, the second half of the original dataset will have its 'median' as the **third quartile**, where 75 % of the values are below it and the rest are above. In case you are wondering, the **second quartile** is really the median of the parent dataset (as explained in the previous point above).<br><br>
    1. The first quartile is marked by the first side of the rectangle, the median by the band within the rectangle and the third quartile by the other side.The **interquartile range (IQR)** is the value of the third quartile minus the first quartile.<br><br>
    1. Outliers were [defined by Tukey as either](http://www.physics.csbsju.edu/stats/box2.html):
        1. outliers that are more than 3 times IQR (3 x IQR) above the third quartile or below the first quartile (marked as filled circles)
        1. suspected outliers that are 1.5 x IQR above the third quartile or below the first quartile (unfilled circles)
       When outliers are present, the whiskers are extended only to the 1.5 x IQR and not the actual minimum and maximum values.
       
       <img src='complex_box_defs.gif' width='300'>
<div align='right'><i>Taken from <a href='http://www.physics.csbsju.edu/stats/box2.html'>Statistics to Use</i></a></div>

The papers by [Lisa Stryjewski (2010)](https://www.semanticscholar.org/paper/40-years-of-boxplots-Stryjewski/d31505d5b6d61ad75a5ae6ded8fa5b7202e66372) and [Hadley Wickham and Lisa Stryjewski (2011)](http://vita.had.co.nz/papers/boxplots.pdf) provide several tools that can complement Tukey's box plot, including as a two-dimensional visualization.

For illustrative purposes, in this notebook, the [Road Accident Deaths in US States dataset](https://vincentarelbundock.github.io/Rdatasets/doc/MASS/road.html) will be used. This dataset contains the annual deaths caused by road accidents for half the US states (26 states). The dataset, in column-separated values (csv) format, will first be imported using the Pandas library and analyzed by various Plotly-based visualizations.

In [6]:
import pandas as pd
df = pd.read_csv('road.csv')

# to make interactive plots with plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly import tools

## <a id='eda'></a>Exploratory Data Analysis (EDA)
We will first look at the data structure and make modifications as required.

In [7]:
df.head() # check to ensure reading the csv file worked as intended

,Unnamed: 0,deaths,drivers,popden,rural,temp,fuel
0,Alabama,968,158,64.0,66.0,62,119.0
1,Alaska,43,11,0.4,5.9,30,6.2
2,Arizona,588,91,12.0,33.0,64,65.0
3,Arkanas,640,92,34.0,73.0,51,74.0
4,Calif,4743,952,100.0,118.0,65,105.0


From above, we can determine that we have 7 columns, with the first being a character/text column and the rest are numeric. 

For some reason, the first column, which contains the names of the states in the United States of America, is not labelled. The other column headers are not really descriptive, so we will rename these first. Additionally, the state 'Arkanas' is a mispelling of Arkansas - probably there are other mistakes as well.

In [9]:
# rename columns based on information from the source webpage (not all needs renaming)
# Columns are:
# 
# state
# deaths - number of deaths.
# drivers - number of drivers (in 10,000s).
# popden - population density in people per square mile.
# rural - length of rural roads, in 1000s of miles.
# temp - average daily maximum temperature in January.
# fuel - fuel consumption in 10,000,000 US gallons per year.

df.rename(columns={'Unnamed: 0': 'State',
                   'deaths': 'Deaths',
                   'drivers': 'Number of drivers (10,000)',
                   'popden': 'Population density (people/mile**2)',
                   'rural': 'Length of rural roads (1000 miles)',
                   'temp': 'Average daily max temperature in Jan',
                   'fuel': 'Fuel consumption (10**7 US gal/year)'
                  }, inplace = True)

Looking at the State column, we can see a mixture of full and abbreviated state names. As not everyone is familiar with the abbreviations, these states will also be renamed. For example, Miss could be interpreted as either Mississippi or Missouri. Fortunately, standardized abbreviations are [available](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

In [10]:
# https://stackoverflow.com/questions/44218091/
df.loc[df['State'] == 'Calif', 'State'] = 'California'
df.loc[df['State'] == 'Colo', 'State'] = 'Colorado'
df.loc[df['State'] == 'Conn', 'State'] = 'Connecticut'

df.loc[df['State'] == 'Dela', 'State'] = 'Delaware'
df.loc[df['State'] == 'DC', 'State'] = 'Washington DC'
df.loc[df['State'] == 'Ill', 'State'] = 'Illinois'
df.loc[df['State'] == 'Ind', 'State'] = 'Indiana'

df.loc[df['State'] == 'Kent', 'State'] = 'Kentucky'
df.loc[df['State'] == 'Louis', 'State'] = 'Louisiana'
df.loc[df['State'] == 'Maryl', 'State'] = 'Maryland'
df.loc[df['State'] == 'Mass', 'State'] = 'Massachusetts'

df.loc[df['State'] == 'Mich', 'State'] = 'Michigan'
df.loc[df['State'] == 'Minn', 'State'] = 'Minnesota'
df.loc[df['State'] == 'Miss', 'State'] = 'Mississippi'
df.loc[df['State'] == 'Mo', 'State'] = 'Missouri'
df.loc[df['State'] == 'Mont', 'State'] = 'Montana'

# State Arkanas is spelled incorrectly
df.loc[df['State'] == 'Arkanas', 'State'] = 'Arkansas'

Before proceeding further, we should inspect the entire dataset first to ensure all the mistakes and column renaming operations above were completed correctly and that there are no missing values.

In [11]:
df

,State,Deaths,"Number of drivers (10,000)",Population density (people/mile**2),Length of rural roads (1000 miles),Average daily max temperature in Jan,Fuel consumption (10**7 US gal/year)
0,Alabama,968,158,64.0,66.0,62,119.0
1,Alaska,43,11,0.4,5.9,30,6.2
2,Arizona,588,91,12.0,33.0,64,65.0
3,Arkansas,640,92,34.0,73.0,51,74.0
4,California,4743,952,100.0,118.0,65,105.0
5,Colorado,566,109,17.0,73.0,42,78.0
6,Connecticut,325,167,518.0,5.1,37,95.0
7,Delaware,118,30,226.0,3.4,41,20.0
8,Washington DC,115,35,12524.0,0.0,44,23.0
9,Florida,1545,298,91.0,57.0,67,216.0


As we can see above, the column labels are more descriptive. Because the dataset is quite small, no memory optimization/downcast operations are needed.

We can now start building a simple box plot.

In [19]:
# all traces are labelled by the State column for easier understanding
# we'll also draw mean lines (dashed)
trace1 = go.Box(y=df.iloc[:,1], marker = dict(size=3), text = df['State'], boxpoints='all', boxmean = True)
trace2 = go.Box(y=df.iloc[:,2], marker = dict(size=3), text = df['State'], boxpoints='all', boxmean = True )
trace3 = go.Box(y=df.iloc[:,3], marker = dict(size=3), text = df['State'], boxpoints='all', boxmean = True )
trace4 = go.Box(y=df.iloc[:,4], marker = dict(size=3), text = df['State'], boxpoints='all', boxmean = True )
trace5 = go.Box(y=df.iloc[:,5], marker = dict(size=3), text = df['State'], boxpoints='all', boxmean = True )
trace6 = go.Box(y=df.iloc[:,6], marker = dict(size=3), text = df['State'], boxpoints='all', boxmean = True )

# we have to make subplots because the scales are different for the variables
fig = tools.make_subplots(rows=2, cols=3, print_grid=False,
                          subplot_titles= (df.columns[1], df.columns[2], df.columns[3],
                                           df.columns[4], df.columns[5], df.columns[6]
                                                          ))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=800, width=800, title='<b>Figure 1: Box Plot of the Road Accidents dataset</b>'
                    )
fig['layout'].update(showlegend=False)

# https://github.com/plotly/plotly.py/issues/985
for i in fig['layout']['annotations']:
    i['font'] = dict(size=10)

for prop in fig.layout:
    if prop.startswith('xaxis'):
        # don't really need both xaxis label and subplot title, so turn xaxis label off
        fig.layout[prop].showticklabels = False

iplot(fig)

Figure 1 shows that for most of the variables, there is at least one point in each that extends beyond the whiskers (outliers). Interestingly, however, the length of rural roads and the average daily max temperature in January are evenly spread (probably normally distributed).

From this, we can make the following observations:
1. The state of California has an extremely high number of deaths compared to the other states.
1. There are significantly more drivers in California and Illinois than the other states, which could partially explain the higher number of deaths.
1. Washington, DC, which is a federal district and the capital of the US, has a significantly high population density. This is probably caused by the small land area with a concentration of foreign diplomats living in the capital.
1. The fuel consumption in Illinois is at the higher extreme end for unknown reasons.
1. With the exception of the population density, all the variables have their average values (mean, dashed line) close to their medians.

The same information can also be tabulated through the Pandas `describe()` function, which can be useful for programmatic access to the descriptive statistics.

In [21]:
# http://www.datasciencemadesimple.com/format-integer-column-of-dataframe-in-python-pandas/
pd.options.display.float_format = '{:,.2f}'.format # round to the 2 decimal places
df.describe()

,Deaths,"Number of drivers (10,000)",Population density (people/mile**2),Length of rural roads (1000 miles),Average daily max temperature in Jan,Fuel consumption (10**7 US gal/year)
count,26.00,26.00,26.00,26.00,26.00,26.00
mean,"1,000.65",191.19,595.73,60.71,43.69,115.24
std,946.84,196.88,"2,437.95",38.38,13.01,83.86
min,43.00,11.00,0.40,0.00,22.00,6.20
25%,571.50,86.50,31.75,30.00,33.75,67.25
50%,799.50,148.50,66.00,65.50,41.50,104.50
75%,"1,265.75",226.25,135.00,93.50,53.25,154.50
max,"4,743.00",952.00,"12,524.00",124.00,67.00,350.00


To determine the correlations between these variables, a scatterplot matrix can be drawn.

In [22]:
# https://community.plot.ly/t/splom-scatter-matrix-changing-styles-of-all-axes-in-one-go/16636/2
# https://plot.ly/python/splom/

# initiate placeholders
dataPanda = []
label = ''

for column in list(df.columns):
        
    columnSplit = column.split(' ') # create a list of string from the column name
    
    for i in columnSplit:
        label = label + i + '<br>' # because plotly uses JavaScript, we can use the html <br> tag to split to next line
    
    # inspired by: https://stackoverflow.com/questions/43229013/
    for i in range(3):
        label = '<br>' + label + '<br>'
    
    label = '<i>' + label + '</i>'
    trace = dict(label = label, values = df[column])
    
    dataPanda.append(trace)
    label = '' # reset the variable for next for iteration

trace1 = go.Splom(dimensions=dataPanda, showupperhalf=False, marker=dict(size=3, color='green'), text = df['State'])
data = [trace1]

layout = go.Layout(title = '<b>Figure 2: Scatterplot Matrix of the Road Accidents dataset</b>',
                font=dict(size=8.5), # global font
                titlefont=dict(size=20), # but we want only title to have different size
                margin = dict(l = 150, r = 20, b = 150) # layout control- left, right, bottom, top (t) and padding (pad )
                )           

fig = go.Figure(data=data, layout=layout)
fig['layout'].update(height=800, width=900)

iplot(fig)

As shown in Figure 2, there are strong positive correlations between the number of deaths and the number of drivers, length of rural roads (presumably these have lower quality surfacing and lighting), average daily maximum temperature in January and fuel consumption. However, it should be noted that strong correlations do not mean causation.

***

## <a id='alternativesToBoxPlots'></a>Alternatives to Box Plots

Although it seems the box plots are useful for summarizing a given dataset, it is by itself insufficient to capture the full content of the dataset.

For instance, a bimodal dataset with many values at 10 and 15 will not be represented as such in a box plot:

<img src='bimodal_bee_swarm.png' width='200'>
<div align='right'><i>Taken from <a href='http://www.physics.csbsju.edu/stats/box2.html'>Statistics to Use</i></a></div>

### Violin Plot

The most directly comparable alternative to box plot is the violin plot. Although rarely used, it is actually more informative than box plots but require some non-intuitive interpretation. Plotly provides a [wrapper function to create violin plots](https://plot.ly/python/violin/), which is useful in explaining the differences with respect to the Road Accidents dataset.

In [24]:
data = []

for i in range(1, len(df.columns)):
    trace = go.Violin(y=df.iloc[:,i], text = df['State'], points = 'all', marker = dict(size=3))
    data.append(trace)

# we have to make subplots because the scales are different for the variables
fig = tools.make_subplots(rows=2, cols=3, print_grid=False,
                          subplot_titles= (df.columns[1], df.columns[2], df.columns[3],
                                           df.columns[4], df.columns[5], df.columns[6]
                                                          ))

fig.append_trace(data[0], 1, 1)
fig.append_trace(data[1], 1, 2)
fig.append_trace(data[2], 1, 3)
fig.append_trace(data[3], 2, 1)
fig.append_trace(data[4], 2, 2)
fig.append_trace(data[5], 2, 3)

fig['layout'].update(height=600, width=800, title='<b>Figure 3: Violin Plot of the Road Accidents dataset</b>')
fig['layout'].update(showlegend=False)

# https://github.com/plotly/plotly.py/issues/985
for i in fig['layout']['annotations']:
    i['font'] = dict(size=11)

for prop in fig.layout:
    if prop.startswith('xaxis'):
        # don't really need both xaxis label and subplot title, so turn xaxis label off
        fig.layout[prop].showticklabels = False

iplot(fig)

Compared to the box plot, in addition to the five-number summary (median, the two quartiles, minimum/maximum, whiskers and outliers), we also obtain the kernal density estimate (kde). 

Violin plots are actually similar to vase plots, except the former use all the data to create the density curve whereas vase plots only use the central 50 %, leading to the presence of whiskers and outliers [ref](https://pdfs.semanticscholar.org/d315/05d5b6d61ad75a5ae6ded8fa5b7202e66372.pdf).

### Maps for Geographical Datasets
Alternatively, for geographical datasets, the use of maps can help in identifying trends. For instance, it could be that the western states have a higher number of deaths than the eastern states.

As the [Plotly choropleth map function](https://plot.ly/python/county-choropleth/) requires the locations to be coded in one of several predefined format, we will need to map the State names to their corresponding codes.

In [10]:
# modified from https://stackoverflow.com/questions/48979352/choropleth-map-in-python-using-plotly-without-state-codes
state_codes = {
    'Washington DC' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 
    'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 
    'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 
    'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 
    'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 
    'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 
    'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 
    'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI',
    'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 
    'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 
    'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 
    'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 
    'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 
    'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 
    'Nevada': 'NV', 'Maine': 'ME'}

df['state_code'] = df['State'].apply(lambda x : state_codes[x])

In [12]:
# df['text'] = df['State'] + '<br>' +\
#         'Deaths ' + df['Deaths'] +' Drivers '+df['Number of drivers (10,000)']+'<br>'+\
#     'Fruits '+df['Population density (people/mile**2)']+' Veggies ' + df['Length of rural roads (1000 miles)']+'<br>'+\
#     'Wheat '+df['Average daily max temperature in Jan']+' Corn '+df['Fuel consumption (10**7 US gal/year)']

# df['text'] = df['State']

# concatenate string with numeric columns:
# https://stackoverflow.com/questions/11858472/pandas-combine-string-and-int-columns
# https://stackoverflow.com/questions/50782934/what-is-difference-between-mapstr-and-astypestr-in-dataframe
# we use PEP8 recommendation to link multi-line codes in parentheses

# df['text'] = ( 
#             df['State'] + '<br>' + 'Deaths: ' + df['Deaths'].astype(str) + 
#             '<br>Drivers: ' + df['Number of drivers (10,000)'].astype(str) +
#             '<br>Population density: ' + df['Population density (people/mile**2)'].astype(str) +
#             '<br>Length of rural road'
#              )

df['text'] = (
            df['State'] + '<br>' + df.columns[1] + ': ' + df.iloc[:, 1].astype(str) +
                          '<br>' + df.columns[2] + ': ' + df.iloc[:, 2].astype(str) +
                          '<br>' + df.columns[3] + ': ' + df.iloc[:, 3].astype(str) +
                          '<br>' + df.columns[4] + ': ' + df.iloc[:, 4].astype(str) +
                          '<br>' + df.columns[5] + ': ' + df.iloc[:, 5].astype(str) +
                          '<br>' + df.columns[6] + ': ' + df.iloc[:, 6].astype(str)
            )

data = [ dict(
        type='choropleth',
       # colorscale = 'Reds',
        autocolorscale = True,
        hoverinfo = 'text', # change from default 'all' so that we don't display to z values on top
        locations = df['state_code'],
        z = df['Deaths'],
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = '<b>Deaths</b>')
        ) ]

layout = dict(
        title = '<b>Figure 4: Choropleth Map of Road Accidents</b><br><i>(Hover for breakdown)</i>',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(0, 0, 255)'), # lakes are in blue
             )
    
fig = dict(data=data, layout=layout)
iplot(fig)

***

## <a id='generationOfSyntheticDataset'></a>Generation of a synthetic dataset

For clarity, each column will be synthesized using one or more standard Scipy distributions and ITS followed by a graphical distribution comparison to the original (experimental) data. In the end, these synthetic columns will be assembled as a Pandas dataframe and corresponding box plots and scatter matrices generated.


***
## <a id='discussion'></a>Discussion

xxx

***
## <a id='conclusion'></a>Conclusion

Experimental dataset are valuable as they capture the nuances of the system under study. However, in many instances, it is not possible to obtain a large number of datapoints, especially when it involves long duration or high expenses. For those situations, it would be useful to generate a set of random values that resemble the experimental data. By having a combined 'experimental + synthetic' dataset, we can gain the ability to train machine learning algorithms, where highly-variant large datasets is a pre-requisite.

Similarly, the use of synthetic datasets helps in the case of parameter estimation for complex (highly parametrized) mechanistic models. Bayesian techniques such as Markov chain Monte Carlo (MCMC) simulations can be performed from a simulated dataset following a verification on the experimental dataset.

Through this notebook, we have learned how the CASP dataset is organized and the different abilities of `numpy.random` and `scipy.stats` libraries as well as their augmentation by inverse transform sampling.